In [3]:
import getpass
import os
import warnings
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langchain.schema import AIMessage, HumanMessage
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing import Literal

from langgraph.prebuilt import ToolNode, tools_condition
from datetime import datetime
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)



llm=ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.5)

In [4]:
@tool
def tool_greet(in_str:str):
    """Tool to be called when user greets"""
    print(f"{in_str}")
    return("Tool called")

@tool
def tool_inquiry(in_str:str):
    """Tool to be called when user inquires about a specific product in the store"""
    print(f"Inquiry started{in_str}")
    return("Inquiry Started")


tools = [tool_greet, tool_inquiry]
llm_with_tools =  llm.bind_tools(tools) 

In [58]:
memory = MemorySaver()

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State):

    # print(llm_with_tools.invoke(state["messages"]))
    print("White Rabbit")
    return {"messages": [llm_with_tools.invoke(state["messages"])]}



In [59]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool_greet, tool_inquiry])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "1"}}

In [66]:
user_input = "Hi there! I am Aditya"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.invoke(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)


White Rabbit
Hi there! I am Aditya
White Rabbit


In [67]:
events['messages'][-1]

AIMessage(content='Hello Aditya! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 290, 'total_tokens': 303, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f04acbbd-4a78-4e51-a7f0-5da2b9ce7277-0', usage_metadata={'input_tokens': 290, 'output_tokens': 13, 'total_tokens': 303, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [68]:
events['messages'][-2]

ToolMessage(content='Tool called', name='tool_greet', id='32d1c17d-edc9-49a8-ac34-23d40b85b477', tool_call_id='call_NJbPO1XjMhGv6hOEpngcgSkg')

In [6]:
tool_inquiry.invoke("Here")

Inquiry startedHere


'Inquiry Started'

In [9]:
kk=eval('tool_inquiry').invoke("Here")

Inquiry startedHere


In [10]:
kk

'Inquiry Started'

In [15]:
kk={'name':'Aditya'}
list(kk.keys())

['name']

In [ ]:
if 'name' in 

In [14]:
output

'"I will be attending a conference on Machine Learning in San Francisco next Monday."'

In [16]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
prompt_template = PromptTemplate(

         input_variables=["query"],
         template=""" Extract all the entities in the query and present them in python dictionary with keys and values.
                      Use lower case letters only
          

          Query: {query}
          
          """
           )
   
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.0)

# llm_chain=prompt_template | llm
# raw_response = llm_chain.invoke(query)['content']

llm_chain= prompt_template | llm
# llm_chain = LLMChain(llm=llm, prompt=prompt_template)


In [19]:
raw_response = llm_chain.invoke("I am visiting New Delhi to eat Pani puri and purchase Nvidia Graphics Card")
raw_response
eval(raw_response.content)

{'location': 'new delhi', 'food': 'pani puri', 'item': 'nvidia graphics card'}

In [19]:
def process_arguments(**kwargs):
    
    if "name" in list(kwargs.keys()):
        print(f"Hello, {kwargs['name']}!")
    
    if "age" in kwargs:
        print(f"Your age is {kwargs['age']}.")
    
    if "location" in kwargs:
        print(f"You are from {kwargs['location']}.")
    
    if not kwargs:
        print("No arguments provided.")

# Example usage:
process_arguments(name="Aditya", age=25)
process_arguments(location="India")
process_arguments()


Hello, Aditya!
Your age is 25.
You are from India.
No arguments provided.


In [23]:
# Example usage:
process_arguments(name="Aditya", age=25)
process_arguments(location="India", name=null)
process_arguments()


Hello, Aditya!
Your age is 25.


NameError: name 'null' is not defined